In [1]:
import torch
from torch import nn
import factorizer as ft

In [8]:
print(12*12*12)

1728


In [42]:
swin_factorizer = ft.Factorizer(
    in_channels=4,
    out_channels=3,
    spatial_size=(8, 8, 8),
    encoder_depth=(1, 1, 1),
    encoder_width=(2, 2, 2),
    strides=(1, 2, 2),
    decoder_depth=(1, 1, 1, 1),
    norm=ft.LayerNorm,
    reshape=(ft.SWMatricize, {'head_dim': 2, 'patch_size': 2}),
    act=nn.ReLU,
    factorize=ft.NMF,
    rank=1,
    num_iters=5,
    init="uniform",
    solver="hals",
    mlp_ratio=2,
    dropout=0.1
)

x = torch.rand((1, 4, 8, 8, 8))

y = swin_factorizer(x)
print("Output shape: ", y.shape)

Output shape:  torch.Size([1, 3, 8, 8, 8])


In [43]:
print(swin_factorizer)

Factorizer(
  (stem): Conv3d(4, 2, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
  (encoder): UNetEncoder(
    (blocks): ModuleList(
      (0): UNetEncoderBlock(
        (downsample): Identity()
        (block): FactorizerStage(
          (pos_embed): Identity()
          (blocks): ModuleList(
            (0): FactorizerBlock(
              (norm1): LayerNorm(
                (norm): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
              )
              (fact): FactMixer(
                (in_proj): Linear(
                  (flatten): Flatten(start_dim=2, end_dim=-1)
                  (linear): Conv1d(2, 2, kernel_size=(1,), stride=(1,), bias=False)
                )
                (reshape): SWMatricize(
                  (shifted_windows): ModuleList(
                    (0-1): 2 x Matricize(
                      (rearrange): Rearrange('b (h d) (g0 p0) (g1 p1) (g2 p2) -> (b h) (g0 g1 g2) d (p0 p1 p2)', d=2, p0=2, p1=2, p2=2)
                    